# Intro to SQL - The Hacker Within (2018-02-21)
This tutorial will show you the basics of SQL:
* creating a table
* inserting rows
* querying with conditions
* grouping and ordering
* joining tables with a common field
* aggregating rows using COUNT, MAX, etc.
* alternates to SQL like pandas and Django
* indexing frequently queried columns for performance

## SQLite
We are going to use [Python](https://docs.python.org/3/library/sqlite3.html) because it comes with the popular [SQLite](https://sqlite.org/index.html) (_aka_ [`etilqs`](https://www.google.com/search?q=etilqs)) relational database builtin. 

## Requirements
The following examples assume you have Python 3 installed on your computer. Any distribution will do. If you have [Anaconda](https://www.anaconda.com/distribution/) or [Miniconda](https://conda.io/miniconda.html), you can create a new conda environment with the requirements for this tutorial from a terminal. For example with miniconda on Mac OS X, you might use the following to activate the root conda environment and create a new one called "py36-thw-sql".

    ~$ source ~/miniconda/bin/activate
    (root) ~$ conda create -n py36-thw-sql python=3.6.3 psycopg2 django pandas

Or on Windows with Anaconda, you might use the following in a CMD terminal.

    ~> %LOCALAPPDATA%\Continuum\anaconda3\Scripts\activate
    (root) ~> conda create -n py36-thw-sql python=3.6.3 psycopg2 django pandas

# Python DB-API 2.0
[PEP 249](https://www.python.org/dev/peps/pep-0249/) specifies a `connection` to a database, and a `cursor` from the connection to execute SQL.

In [1]:
# imports
import sqlite3  # this is the module that binds to SQLite
import os

DBFILE = 'sqlite3.db'  # this will be our database
BASEDIR = %pwd  # os.path.abspath(os.path.dirname(__file__))
DBPATH = os.path.join(BASEDIR, DBFILE)
# we may need to delete the existing file first
if os.path.exists(DBPATH):
    os.remove(DBPATH)

# open a connection to the database for this tutorial
conn = sqlite3.connect(DBPATH)

# get a cursor to the database
cur = conn.cursor()

## Notes
If a file with the same name doesn't already exist, then this creates a new database otherwise it connects to the existing database contained in the file. You can also use ':memory:' to create an "in-memory" database that has no file, but then you can't connect to that from another process.

We'll have to close the connection and cursor later. Next time we could use a [`with` context to automatically close the connection](https://docs.python.org/3.6/library/sqlite3.html#using-the-connection-as-a-context-manager).

    with sqlite3.connect('sqlite3.db') as conn:
        cur = conn.execute('SQL QUERY ...')  # e.g.: 'SELECT * FROM table_name;'
        output = cur.fetchall()  # get the results

Closing the connection automatically closes the cursor. [Other bindings may offer similar context managers.](http://initd.org/psycopg/docs/usage.html#with-statement) to commit and close changes or rollback changes and raise an exception.

# Creating tables
A relational database or SQL database is a tabular structure consisting of rows of data with columns of fields. The [data definition language or DDL](https://en.wikipedia.org/wiki/Data_definition_language) used to create the table is the same language used to query it, called [SQL or Structured Query Language](https://en.wikipedia.org/wiki/SQL).

Although [the basic SQL commands](https://sqlite.org/lang.html) are nearly the same for [other relational databases](https://www.postgresql.org/docs/10/static/sql.html), the data types may be different. [SQLite only has 5 datatypes](https://sqlite.org/datatype3.html): `NULL`, `INTEGER`, `REAL`, `TEXT`, `BLOB`. For [boolean, use integer zero for false, and one for true](https://sqlite.org/datatype3.html#boolean_datatype). For [dates and times use text and ISO8601](https://sqlite.org/datatype3.html#date_and_time_datatype), _e.g._: `"2018-02-21T17:05-0800"`. By comparison, [PostgreSQL has too many to list here](https://www.postgresql.org/docs/10/static/datatype.html) including booleans, date, time, arrays, JSON, etc.

## `CREATE`
The [basic SQL command to create a table](https://sqlite.org/lang_createtable.html) is

    CREATE TABLE <table_name> (<field_name> <TYPE> <CONSTRAINTS>, <field_name> <TYPE> <CONSTRAINTS>, <CONSTRAINTS>, ...);

Some languages enforce the semicolon, some don't. The syntax is nearly the same for [other relational databases](https://www.postgresql.org/docs/10/static/ddl-basics.html).

### Constraints, Defaults, and Options
Constraints are optional and set conditions, limitations, or options for columns and the table. The most common constraints are: `PRIMARY KEY`, `UNIQUE`, `NOT NULL`, `DEFAULT`, `FOREIGN KEY`, `REFERENCES`, etc. The syntax is nearly the same for [other relational databases](https://www.postgresql.org/docs/10/static/ddl-constraints.html).

#### `PRIMARY KEY`
The most important of these is `PRIMARY KEY` which is equivalent to `UNIQUE NOT NULL`. A [primary key](https://www.postgresql.org/docs/10/static/ddl-constraints.html#DDL-CONSTRAINTS-PRIMARY-KEYS) is a unique references that identifies each record in the table. Although it is not required, every table should have a primary key. Only one primary key is allowed, and it can be constructed from multiple columns, `PRIMARY KEY (<field_A>, <field_B)`, to create a unique together, non-null identifier. [In SQLite, if missing then a integer primary key named, `rowid`, is created by default](https://sqlite.org/lang_createtable.html#rowid). Also in SQLite, any integer primary key is automatically incremented, so [the _AUTOINCREMENT_ command is usually **not** needed](https://sqlite.org/autoinc.html). [In PostgreSQL the `SERIAL` command is used](https://www.postgresql.org/docs/10/static/datatype-numeric.html#DATATYPE-SERIAL) to create a corresponding [sequence](https://www.postgresql.org/docs/10/static/functions-sequence.html) for the primary key.

## Practice
The other constraints and options are also important, but we'll discover those as we learn. Let's create some simple databases with fictitious data to practice. Imagine you are testing several different materials with different properties $\alpha$ and $\beta$ under different stresses like different temperatures and  light intensity and changing thickness. How would you organize this data? Take a moment to design a schema or structure for your data. The schema consists of the column names and data types and the column and table constraints.

In [2]:
# we can use Python triple quoted strings to span multiple lines, but use single quotes, since SQL only uses double quotes

# first create a materials table
cur.execute('''CREATE TABLE materials (
    material_id INTEGER PRIMARY KEY,
    name TEXT,
    alpha REAL NOT NULL,
    beta REAL NOT NULL
)''')
conn.commit()
# if you don't commit the changes, they won't be written to the file, and won't be visible to other connections

# then create an experiments table
cur.execute('''CREATE TABLE experiments (
    experiment_id INTEGER PRIMARY KEY,
    temperature REAL DEFAULT 298.15,
    irradiance REAL DEFAULT 1000.0,
    uv_filter INTEGER DEFAULT 0,
    material_id NOT NULL REFERENCES materials ON UPDATE CASCADE ON DELETE CASCADE,
    thickness REAL DEFAULT 0.005,
    UNIQUE (temperature, irradiance)
)''')
conn.commit()

# and finally create a trials table
cur.execute('''CREATE TABLE trials (
    trial_id INTEGER PRIMARY KEY,
    experiment_id NOT NULL REFERENCES experiments ON UPDATE CASCADE ON DELETE CASCADE,
    results BLOB NOT NULL,
    duration REAL NOT NULL,
    avg_temperature REAL NOT NULL,
    std_temperature REAL NOT NULL,
    avg_irradiance REAL NOT NULL,
    std_irradiance REAL NOT NULL,
    init_visible_transmittance REAL NOT NULL,
    final_visible_transmittance REAL NOT NULL,
    notes TEXT
)''')
conn.commit()

## `INSERT`
The [basic SQL command to put data into a table is](https://sqlite.org/lang_insert.html)

    INSERT INTO <table_name> (<field_name>, <field_name>, ...) VALUES (<value>, <value>, ...)

[Other relational databases use the same SQL syntax.](https://www.postgresql.org/docs/current/static/dml-insert.html)

# Queries
The way you select data is by executing queries.

In [ ]:
cur.close()
conn.close()